# SISTEMAS DE BIG DATA - Examen 1ª Evaluación

**Instrucciones generales**

1.	Todas las sentencias deben ejecutarse desde la línea de comandos en las celdas que hay después del enunciado. No debes realizar ninguna tarea desde fuera de Jupyter.
2.	Puedes **añadir** todas las celdas que necesites siempre y cuando estén antes del siguiente enunciado.
3.	Todas las celdas **deben estar ejecutadas** y debe visualizarse el resultado de salida.
4.	**No es necesario documentar** las respuestas, simplemente debes hacer lo que se pide en el enunciado.
5.	Después de cada parte debes insertar una **captura de pantalla** del cliente gráfico de la base de datos correspondientes donde se vea que los datos se han cargado correctamente.
6.	Debes entregar tanto el **notebook** (fichero `.ipynb`) como el mismo fichero convertido a **PDF** (es muy probable que si intentas convertirlo en el propio contenedor te falle por no tener instalado `pandoc`, si es así descargalo en formato `.md` o `html` y conviértelo en tu máquina física).

---

**NOMBRE**:

---

In [2]:
!pip install influxdb-client
!pip install psutil

## Contexto del escenario

Has sido contratado por una fábrica inteligente que dispone de sensores de temperatura y vibración en sus máquinas críticas. La empresa necesita un sistema backend capaz de procesar los datos que llegan de los sensores en tiempo real.

El sistema debe cumplir dos objetivos simultáneos:

1.  **Monitorización en vivo (Dashboard):** los operarios necesitan saber el estado *actual* de cada máquina y si hay alguna alarma activa en este preciso instante. Para esto usarás **Redis**.
2.  **Histórico para mantenimiento predictivo:** el equipo de Data Science necesita almacenar todos los datos brutos a lo largo del tiempo para entrenar modelos de IA futuros. Para esto usarás **InfluxDB**.

## Los Datos de Entrada

Los datos con los que vas a trabajar los tienes en el *dataset* sintético adjunto llamado `sensores.csv`. Este *dataset* contiene lecturas simuladas con las siguientes columnas:

  - `timestamp`: fecha y hora del evento.
  - `machine_id`: identificador único de la máquina.
  - `zone`: zona de la fábrica.
  - `temperature`: temperatura en grados Celsius.
  - `vibration`: nivel de vibración (0-100).
  - `lat`, `lon`: coordenadas del robot.
  - `status`: estado reportado por la máquina ("OK", "WARNING", "ERROR").

**IMPORTANTE**

El desarrollo del examen debe de ser modular, con un programa principal que inicialice las conexiones a la base de datos y lea los datos del fichero y luego invocará **una función diferente para cargar cada tipo de dato** en la base de datos

Es decision tuya elegir los parámetros que recibirá cada función, aunque es altamente aconsejable **no utilizar variables globales**.

## Parte A: Persistencia histórica (InfluxDB)

`2 puntos`

En esta parte tienes que crear un script que lea el fichero CSV facilitado y almacene los datos en una base de datos InfluxDB.

Los aspectos que tienes que tener en cuenta son:

  - **Bucket:** `factory_logs`
  - **Measurement:** `maquinaria`
  - **Requisito clave:** debes modelar correctamente los datos usando adecuadamente *tags* o *fields* según el tipo de datos. Se debe respetar el `timestamp` del datos (no usar el tiempo de ingesta).

In [7]:
import psutil
import time
import csv
import sys
import influxdb_client
from datetime import datetime
from influxdb_client import Point, InfluxDBClient
from influxdb_client.client.write_api import WriteOptions, WriteType, ASYNCHRONOUS
from influxdb_client.client.exceptions import InfluxDBError
from urllib3.exceptions import NewConnectionError

puntos_fichero = 0
TIME_FORMAT = '%Y-%m-%d %H:%M:%S'

INFLUX_URL = "http://influxdb2:8086"
INFLUX_TOKEN = "pY8NneFC5iDvPM2LKuL1v-e1NR7was3ddvW5g6jniZ30voUqkoRvEIg74VBwioQowBIGHh-4FjPy60lW24e1vQ=="
INFLUX_ORG = "docs"
BUCKET_NAME = "factory_logs"
FILE_NAME = "telemetria_agv.csv"

MEASUREMENT_NAME = "maquinaria"

client = InfluxDBClient(url=INFLUX_URL, token=INFLUX_TOKEN, org=INFLUX_ORG)

def success_callback(conf, data):
    """Callback al finalizar un batch con éxito."""
    pass 

def error_callback(conf, data, exception):
    """Callback para manejar errores de escritura."""
    print(f"\n--- ERROR DE ESCRITURA ---")
    print(f"Excepción: {exception}")
    print(f"--------------------------\n")

write_api = client.write_api(
        write_options=ASYNCHRONOUS,
        success_callback=success_callback,
        error_callback=error_callback
    )

with open(FILE_NAME, mode='r', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)

    next(reader, None)

    for row in reader:
        if len(row) < 8:
            continue
    
        timestamp_str = row[0]
        timestamp_dt = datetime.strptime(timestamp_str, TIME_FORMAT)
        
        machine_id = row[1]
        zone = row[2]
        temperature = float(row[3])
        vibration = float(row[4])
        lat = float(row[5])
        lon = float(row[6])
        status = row[7]
    
        point = (
            Point(MEASUREMENT_NAME)
            .tag("machine_id", machine_id)
            .time(timestamp_dt) 
            .field("zone", zone)
            .field("temperature", temperature)
            .field("vibration", vibration)
            .field("lat", lat)
            .field("lon", lon)
            .field("status", status)
        )
        
        write_api.write(bucket=BUCKET_NAME, record=point)
    
        puntos_fichero += 1
                        
        print(f"   -> {puntos_fichero} puntos procesados", end='\r')


    print(f"\nProcesamiento de {puntos_fichero} puntos finalizado.")

# Asegúrate de cerrar la API para vaciar el buffer
write_api.close()
client.close()
    

SyntaxError: invalid syntax (2050943277.py, line 82)

## Parte B - Analítica en tiempo real con Redis

Debes crear un script que alimente las siguientes estructuras en Redis por cada dato procesado:

In [28]:
!pip install redis

### 1.- Estadísticas agregadas

`1 punto`

Al procesar masivamente datos de telemetría, es costoso consultar la base de datos histórica (InfluxDB) para preguntas simples como "¿Cuál ha sido la temperatura máxima hoy en el Almacén A?". Vamos a usar Redis Hashes para mantener un marcador actualizado de estadísticas por zona.

Para cada fila procesada del CSV, debes actualizar un Hash correspondiente a la Zona (zone) donde se encuentra el robot.

- **Clave:** `stats:zone:{nombre_zona}` (Ej: stats:zone:Almacen_A, stats:zone:Recepcion...).
- **Campos:**:
    - `total_lecturas`: contador total de datos recibidos de esa zona.
    - `total_errores`: contador de cuántas veces el status ha sido "ERROR".
    - `max_temp`: La temperatura más alta registrada hasta el momento en esa zona.

In [20]:
import redis

r = redis.Redis(
    host='redis',
    port=6379,
    db=0,
    decode_responses=True
)
print(r.ping())

True


In [33]:
reader = csv.reader(sys.stdin)

max_tempA = 0
max_tempB = 0
max_tempR = 0
max_tempEn = 0
max_tempEx = 0

lecA = 0
lecB = 0
lecR = 0
lecEn = 0
lecEx = 0

errA = 0
errB = 0
errR = 0
errEn = 0
errEx = 0

for row in reader:

    if row[1] == "machine_id":
        continue
    try:
        zone = row[2]
        status = row[7]
        temp = row[3]
    except ValueError:
         continue

    if zone == "Almacen_A" :
        lecA += 1
        if status == "ERROR":
            errA +=1
        if temp> max_tempA:
            max_tempA = temp

    if zone == "Almacen_B" :
        lecB += 1
        if status == "ERROR":
            errB +=1
        if temp> max_tempB:
            max_tempB = temp

    if zone == "Recepcion" :
        lecR += 1
        if status == "ERROR":
            errR +=1
        if temp> max_tempR:
            max_tempR = temp

    if zone == "Ensamblaje" :
        lecEn += 1
        if status == "ERROR":
            errEn +=1
        if temp> max_tempEn:
            max_tempEn = temp

    if zone == "Expediciones" :
        lecEx += 1
        if status == "ERROR":
            errEx +=1
        if temp> max_tempEx:
            max_tempEx = temp

def add_zone(zone, total_lecturas, total_errores, max_temp):
    clave_zona = f"stats:zone:{zone}"
    
    Hzone= {
        "lecturas" : total_lecturas,
        "errores" : total_errores,
        "max_temp" : max_temp
    }

    r.hset(clave_zona, mapping=Hzone)
    
    print(f"Zona '{clave_zona}' añadido.")

add_zone("Almacen_A", lecA, errA, max_tempA)
add_zone("Almacen_B", lecB, errB, max_tempB)
add_zone("Recepcion", lecR, errR, max_tempR)
add_zone("Ensamblaje", lecEn, errEn, max_tempEn)
add_zone("Expediciones", lecEx, errEx, max_tempEx)

Zona 'stats:zone:Almacen_A' añadido.
Zona 'stats:zone:Almacen_B' añadido.
Zona 'stats:zone:Recepcion' añadido.
Zona 'stats:zone:Ensamblaje' añadido.
Zona 'stats:zone:Expediciones' añadido.


### 2.- Ranking de "puntos calientes" (Sorted Set)

`1 punto`

El jefe de planta quiere ver en una pantalla un "Top de Máquinas con mayor temperatura" ordenado de mayor a menor en tiempo real.

- **Estructura:** `Sorted Set` (ZSET)
- **Clave:** `dashboard:hottest_machines`
- **Score:** La temperatura actual (`temperature`).
- **Member:** El ID de la máquina (`machine_id`).

In [ ]:
# Función que carga el sorted set


### 3.- Seguimiento de flota (Geospatial)

`1 punto`

Las máquinas de este escenario son AGVs (robots móviles) que se mueven por la planta. Necesitamos saber su ubicación exacta.

- **Estructura:** `Geo`
- **Clave:** `factory:map`
- **Datos:** Usa la latitud y longitud que vienen en el CSV para posicionar el `machine_id`.

In [ ]:
# Función que carga los datos geoespaciales


### 4.- Contadores globales atómicos (String)

`1 punto`

Necesitamos estadísticas rápidas que no requieran contar filas en una base de datos histórica.

- **Estructura:** `String` (Contador)
- **Clave:** `stats:total_processed` -\> Incrementar en 1 por cada fila procesada.
- **Clave:** `stats:total_errors` -\> Incrementar en 1 solo si el `status` es "ERROR".
- **Clave:** `stats:total_warnings` -\> Incrementar en 1 solo si el `status` es "WARNING".



In [ ]:
# Función que gestiona los contadores


### 5.- Cola de anomalías críticas (List)

`1 punto`

Queremos tener también una cola de anomalías críticas. Por cada registro cuyo `status` sea `ERROR` deberás crear un JSON y almacenarlo en una estructura tipo FIFO:

- **Estructura:** `List`
- **Clave:** `alerts:queue`
- **Datos:**: el JSON debe incluir: `machine_id`, `timestamp` y un mensaje: *"Critical failure at [Lat, Lon]"*.

In [ ]:
# Función que carga los datos en la cola


## Programa principal

In [ ]:
# Aquí debes insertar el programa principal que llama al resto de funciones


## Capturas de pantalla

A partir de aquí tienes que insertar las capturas de pantalla correspondientes a cada punto. Las capturas de pantalla corresponderán a la interfaz gráfica de la base de datos correspondiente y se debe mostrar que los datos se han cargado correctamente. Los apartados que no tengan la captura de pantalla correspondiente **se considerarán no realizados**.

### Captura de InfluxDB

### Captura de estadísticas agregadas

### Captura de ranking de puntos calientes

### Captura de seguimiento de flota

### Captura de contadores globales atómicos

### Captura de cola de anomalías críticas